In [1]:
import pandas as pd

###  Loading data of books we like

In [2]:
#loading our liked books data
my_books = pd.read_csv("liked_books.csv")

#typecasting book_id into string
my_books["book_id"] = my_books["book_id"].astype(str)

#creating set of unique book from our liked_books data
book_set = set(my_books["book_id"])
books = len(book_set)

#  Creating Dictionary from book_mapping dataset to create connection between two dataset

In [3]:
#creating dictionary from csv_book_mapping to connect both dataset based on this

csv_book_mapping = {}

with open("book_id_map.csv",'r') as f:
    while True:
        line = f.readline()
        
        if not line:
            break;
            
        csv_id,book_id = line.strip().split(",")
        
        csv_book_mapping[csv_id] = book_id


## Getting all the users who have rated books which we like

In [4]:
# finding all users who has read books which we have liked and rated >= 4 starts.
overlap_users = {}

#reading goodreads_interactions data in a streaming fashion
with open("goodreads_interactions.csv") as f:
    i = 0
    while i < 1000:
        i +=1 
        
        #reading single line
        line = f.readline()
        if not line:
            break;
        
        # splitting data by ',' and getting all the columns we care about
        user_id, csv_id, _, rating, _ = line.strip().split(",")
        
        #converting rating into int if not possible ignore that line
        try:
            rating = int(rating)
        except ValueError:
            continue
            
        
        book_id = csv_book_mapping.get(csv_id)
        
        
        if book_id in book_set and rating >= 4:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1
                

In [5]:
filtered_overlap_users = set(i for i in overlap_users if overlap_users[i] > books/2)

### finding records of book ratings where only those users have rated, who has read all the books which we like


In [6]:
#finding records of book ratings where only those users have rated, who has read all the books which we like
#creating records using overlap_users data

rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
# with open("new.csv", 'r') as f:
    i = 0
#     while i < 1000:
    while i<10000000:
        i += 1
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            if book_id not in book_set:
                rec_lines.append([user_id, book_id, rating])
        

In [17]:
#rec_lines

###  Loading all those records into pandas dataframe

In [8]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id","book_id","rating"])
#recs["book_id"] = recs["book_id"].astype(str)

In [20]:
recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   book_id      0 non-null      object
 1   cover_image  0 non-null      object
 2   ratings      0 non-null      int64 
 3   title        0 non-null      object
 4   url          0 non-null      object
dtypes: int64(1), object(4)
memory usage: 108.0+ bytes


In [22]:
recs

,book_id,cover_image,ratings,title,url


###  Getting top 15 books which is rated by highest number of users who rated books we likes

In [10]:
top_recs = recs["book_id"].value_counts().head(15)
# top_recs = top_recs.index.to_list()


In [23]:
top_recs

12          1
8622058     1
13531802    1
3977        1
15793509    1
6486483     1
15801967    1
10294362    1
16071764    1
2819368     1
143718      1
349419      1
809849      1
16033093    1
99219       1
Name: book_id, dtype: int64

###  loading data from books_titles.json files and printing all the top records data from this json file

In [12]:
book_titles = pd.read_json("books_titles.json")
book_titles["book_id"] = book_titles["book_id"].astype(str)

In [13]:
def clickable(val):
    return '<a target="_blank" href="{}">book Link</a>'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

In [14]:
book_titles.head(10).style.format({'url':clickable, 'cover_image' : show_image})

,book_id,cover_image,ratings,title,url
0,1333909,,10,good harbor,book Link
1,7327624,,140,the unschooled wizard sun wolf and starhawk 12,book Link
2,6066819,,51184,best friends forever,book Link
3,287140,,15,runic astrology starcraft and timekeeping in the northern tradition,book Link
4,287141,,46,the aeneid for boys and girls,book Link
5,378460,,12,the wanting of levine,book Link
6,6066812,,98,alls fairy in love and war avalon web of magic 8,book Link
7,287149,,986,the devils notebook,book Link
8,6066814,,186,crowner royal crowner john mystery 13,book Link
9,33394837,,269,the house of memory plutos snitch 2,book Link


In [15]:
recs = book_titles[book_titles["book_id"].isin(top_recs)] 
len(recs)

0

# top recommended books for us

In [16]:
recs.style.format({'url':clickable, 'cover_image' : show_image})

,book_id,cover_image,ratings,title,url
